<a href="https://colab.research.google.com/github/cifkao/ss-vq-vae/blob/main/experiments/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Timbre transfer demo

Copyright 2020 InterDigital R&D and Télécom Paris.  
Author: Ondřej Cífka

## Download the model

In [1]:
logdir = 'model'
logdir_original = 'model-original'

In [2]:
!wget https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/ssvqvae_model_state.pt -O $logdir_original/model_state.pt

--2023-08-07 17:23:37--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/ssvqvae_model_state.pt
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222788899 (212M)
Saving to: ‘model-original/model_state.pt’

model-original/mode 100%[===================>] 212.47M  27.0MB/s    in 7.4s    

2023-08-07 17:23:45 (28.8 MB/s) - ‘model-original/model_state.pt’ saved [222788899/222788899]



## Load the model

In [2]:
import sys
# sys.path.append('/home/wk406185/PSP/ss-vq-vae')
# sys.path.append('/home/wk406185/PSP/ss-vq-vae/src')
# sys.path.append('/home/wk406185/PSP/ss-vq-vae/src/ss_vq_vae')

In [3]:
import os

import confugue
# from ddsp.colab import colab_utils
import librosa
import torch

from ss_vq_vae.models.vqvae_oneshot import Experiment, ExperimentZeroShot

2023-09-13 20:13:59.729453: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 20:14:00.851743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 20:14:03.845180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load the zero-shot model

In [7]:
logdir = '/mnt/vdb/model-leaky-relu-unfrozen-style-encoder-11-09-2023-2/'
cfg = confugue.Configuration.from_yaml_file(os.path.join(logdir, 'config-zero-shot.yaml'))
exp = cfg.configure(ExperimentZeroShot, logdir=logdir, device='cuda', pretrained_cola=False, continue_training=False, frozen_style_encoder=False, continue_step=0)
exp.model.load_state_dict(torch.load(os.path.join(logdir, 'model_state.pt'), map_location=exp.device))
exp.model.train(False)

ModelZeroShotUnfrozenStyle(
  (content_encoder): Sequential(
    (0): Conv1d(1025, 1024, kernel_size=(4,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv1d(1024, 1024, kernel_size=(4,), stride=(2,), padding=(2,))
    (4): ResidualWrapper(
      (module): Sequential(
        (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): LeakyReLU(negative_slope=0.1)
        (2): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
      )
    )
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (vq): VQEmbedding(
    (embedding): Embedding(2048, 1024)
  )
  (style_encoder): COLA(
    (encoder): EfficientNetEmbedding(
      (backend): EfficientNet(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), paddin

In [8]:
INPUT_ROOT = 'https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/'
INPUT_URLS = {
    'Electric Guitar': INPUT_ROOT + 'real/content/UnicornRodeo_Maybe_UnicornRodeo_Maybe_Full_25_ElecGtr2CloseMic3.0148.mp3',
    'Electric Organ': INPUT_ROOT + 'real/style/AllenStone_Naturally_Allen%20Stone_Naturally_Keys-Organ-Active%20DI.0253.mp3',
    'Jazz Piano': INPUT_ROOT + 'real/style/MaurizioPagnuttiSextet_AllTheGinIsGone_MaurizioPagnuttiSextet_AllTheGinIsGone_Full_12_PianoMics1.08.mp3',
    'Synth': INPUT_ROOT + 'real/content/Skelpolu_TogetherAlone_Skelpolu_TogetherAlone_Full_13_Synth.0190.mp3'
}

## Choose or record inputs

In [9]:
import IPython.display as ipd
import librosa

#@title Content input
content_input = 'Electric Guitar'  #@param ["Record", "Electric Guitar", "Electric Organ", "Jazz Piano", "Synth"]
record_seconds = 8 #@param {type:"number"}

if content_input == 'Record':
    a_content = colab_utils.record(seconds=record_seconds, sample_rate=exp.sr, normalize_db=0.1)
else:
    !wget {INPUT_URLS[content_input]} -O content_input.mp3
    a_content, _ = librosa.load('content_input.mp3', sr=exp.sr)
# colab_utils.play(a_content, sample_rate=exp.sr)
ipd.Audio(a_content, rate=exp.sr)

--2023-09-13 20:16:36--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/real/content/UnicornRodeo_Maybe_UnicornRodeo_Maybe_Full_25_ElecGtr2CloseMic3.0148.mp3
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162004 (158K) [audio/mpeg]
Saving to: ‘content_input.mp3’

content_input.mp3   100%[===================>] 158.21K  --.-KB/s    in 0.08s   

2023-09-13 20:16:36 (1.86 MB/s) - ‘content_input.mp3’ saved [162004/162004]



In [10]:
#@title Style input
style_input = 'Jazz Piano'  #@param ["Record", "Electric Guitar", "Electric Organ", "Jazz Piano", "Synth"]
record_seconds = 8 #@param {type:"number"}

if style_input == 'Record':
    a_style = colab_utils.record(seconds=record_seconds, sample_rate=exp.sr, normalize_db=0.1)
else:
    !wget {INPUT_URLS[style_input]} -O style_input.mp3
    a_style, _ = librosa.load('style_input.mp3', sr=exp.sr)
# colab_utils.play(a_style, sample_rate=16000)
ipd.Audio(a_style, rate=exp.sr) 

--2023-09-13 20:16:42--  https://adasp.telecom-paris.fr/rc-ext/demos_companion-pages/vqvae_examples/real/style/MaurizioPagnuttiSextet_AllTheGinIsGone_MaurizioPagnuttiSextet_AllTheGinIsGone_Full_12_PianoMics1.08.mp3
Resolving adasp.telecom-paris.fr (adasp.telecom-paris.fr)... 137.194.22.227, 2a04:8ec0:0:a::89c2:16e3
Connecting to adasp.telecom-paris.fr (adasp.telecom-paris.fr)|137.194.22.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162004 (158K) [audio/mpeg]
Saving to: ‘style_input.mp3’

style_input.mp3     100%[===================>] 158.21K  --.-KB/s    in 0.08s   

2023-09-13 20:16:43 (1.87 MB/s) - ‘style_input.mp3’ saved [162004/162004]



## Run the model

In [11]:
s_content = torch.as_tensor(exp.preprocess(a_content), device=exp.device)[None, :]
s_style = torch.as_tensor(exp.melspectrogram_preprocess(a_style, 16_000), device=exp.device)[None, :]
l_content, l_style = (torch.as_tensor([x.shape[2]], device=exp.device) for x in [s_content, s_style])


In [12]:
s_style.shape

torch.Size([1, 64, 801])

In [13]:
import numpy as np

# with torch.no_grad():
#     s_output = exp.model(input_c=s_content, input_s=s_style,
#                          length_c=l_content, length_s=l_style, output_c=None, output_l=None)
with torch.no_grad():
    s_output = exp.model(input_c=s_content, input_s=s_style,
                         length_c=l_content, length_s=l_style, output_c=None, output_l=None)
# s_output
x = s_output[0].cpu().numpy()
print(x.shape)
a_output = exp.postprocess(s_output[0].cpu().numpy())
a_output /= a_output.max()
# colab_utils.play(a_output, sample_rate=16000)
ipd.Audio(a_output, rate=16_000)
# np.isnan(a_output).any()

(1025, 257)


In [26]:
encoded_c, encoded_s = s_output[1:]
length = l_content
max_length = s_content.shape[2]

with torch.no_grad():
    encoded_s = encoded_s[:, :, None]

    decoded = encoded_c
    for module in exp.model.decoder_modules[:-1]:
        print("Decoded: ", decoded)
        decoded = torch.cat([
            decoded,
            encoded_s.expand(-1, -1, decoded.shape[-1])
        ], axis=1)
        decoded = module(decoded)
        
    decoded = torch.cat([
        decoded,
        encoded_s.expand(-1, -1, decoded.shape[-1])
    ], axis=1)
    for name, module in exp.model.decoder_modules[-1]._modules.items():
        decoded = module(decoded)
        print("After last decoder module: ", name, decoded)
    print("Decoded: ", decoded)

    # Make sure the output tensor has the same shape as the input tensor
    if max_length is not None or length is not None:
        if max_length is None:
            max_length = length.max()

        decoded = decoded.narrow(-1, 0, max_length)

    # Make sure output lengths are the same as input lengths
    if length is not None:
        mask = (torch.arange(max_length, device=decoded.device) < length[:, None])[:, None, :]
        decoded = decoded * mask

    decoded
decoded

Decoded:  tensor([[[ 0.4816,  0.4816,  0.4816,  ...,  0.4816,  0.4816,  0.4816],
         [-0.4864, -0.4864, -0.4864,  ..., -0.4864, -0.4864, -0.4864],
         [-0.5477, -0.5477, -0.5477,  ..., -0.5477, -0.5477, -0.5477],
         ...,
         [ 0.5038,  0.5038,  0.5038,  ...,  0.5038,  0.5038,  0.5038],
         [ 0.4978,  0.4978,  0.4978,  ...,  0.4978,  0.4978,  0.4978],
         [-0.4958, -0.4958, -0.4958,  ..., -0.4958, -0.4958, -0.4958]]],
       device='cuda:0')
After last decoder module:  0 tensor([[[ 2.3337, -5.8915,  2.3332,  ...,  2.3324, -5.8914,  1.1780],
         [-5.1546,  4.3964, -5.1550,  ..., -5.1554,  4.3945, -2.5566],
         [-1.7423,  2.5840, -1.7422,  ..., -1.7410,  2.5831, -1.3282],
         ...,
         [ 3.7252,  3.7252,  3.7252,  ...,  3.7252,  3.7252,  3.7252],
         [-5.6504, -5.6504, -5.6504,  ..., -5.6504, -5.6504, -5.6504],
         [ 3.1959,  3.1959,  3.1959,  ...,  3.1959,  3.1959,  3.1959]]],
       device='cuda:0')
After last decoder module:  

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')

# Calculate LSD on random_audios_data

## Load data

In [14]:
import pandas as pd

In [15]:
ground_audios_df = pd.read_csv("../data/random_audios/generated_files.csv")
ground_audios_df.head()

,content_file,style_file,ground_file
0,67/data_029_067.wav,43/data_009_043.wav,43/data_029_043.wav
1,17/data_052_017.wav,67/data_059_067.wav,67/data_052_067.wav
2,45/data_087_045.wav,92/data_094_092.wav,92/data_087_092.wav
3,17/data_087_017.wav,43/data_042_043.wav,43/data_087_043.wav
4,34/data_069_034.wav,98/data_024_098.wav,98/data_069_098.wav


In [23]:
with open("outputs/random_audios/vqvae_list") as f:
    generated_audios_list = f.read().splitlines()
generated_audios_list[:10]

['outputs/random_audios/vqvae000.wav',
 'outputs/random_audios/vqvae001.wav',
 'outputs/random_audios/vqvae002.wav',
 'outputs/random_audios/vqvae003.wav',
 'outputs/random_audios/vqvae004.wav',
 'outputs/random_audios/vqvae005.wav',
 'outputs/random_audios/vqvae006.wav',
 'outputs/random_audios/vqvae007.wav',
 'outputs/random_audios/vqvae008.wav',
 'outputs/random_audios/vqvae009.wav']

In [18]:
import IPython.display as ipd

HOME_ROOT = "/home/user/random_audios_grouped/random_audios_test_16k"
rate = 16_000

content_path, style_path, reference_path = ground_audios_df.iloc[0]

ipd.Audio(f"{HOME_ROOT}/{content_path}", rate=rate)

In [25]:
ipd.Audio(f"{HOME_ROOT}/{style_path}", rate=rate)

In [26]:
ipd.Audio(f"{HOME_ROOT}/{reference_path}", rate=rate)

In [27]:
GENERATED_ROOT = "/home/user/ss-vq-vae/experiments/outputs/random_audios/"
generated_sample = generated_audios_list[0]

ipd.Audio(f"{GENERATED_ROOT}/{generated_sample}", rate=rate)

In [28]:
assert len(generated_audios_list) == len(ground_audios_df)

## Calculate LSD

In [16]:
import os
import librosa
import numpy as np

def normalize_power(audio):
    return audio / (np.sqrt(np.mean(audio ** 2)) + np.finfo(audio.dtype).eps)

def eval_example_target(output, reference):
    def spec(audio):
        audio = normalize_power(audio)
        s = librosa.feature.melspectrogram(y=audio, sr=rate)
        return librosa.power_to_db(s)
    
    s_out, s_ref = spec(output), spec(reference)
    lsd = np.mean(np.sqrt(np.mean((s_out - s_ref) ** 2, axis=1)))

    return {'lsd': lsd}

### Calculate the original model's LSD

In [30]:
import os

lsds = []
for (_, (_, _, ground_path)), generated_path in zip(ground_audios_df.iterrows(), generated_audios_list):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])

print(sum(lsds) / len(lsds))

90.93506548242661


### Calculate Unfrozen style encoder model's LSD (`model-leaky-relu-30-08-2023`)

In [32]:
import os

GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/random_audios_30-08-2023/"
with open("outputs/random_audios_30-08-2023/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
for (_, (_, _, ground_path)), generated_path in zip(ground_audios_df.iterrows(), generated_custom_audios_list):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])

print(sum(lsds) / len(lsds))

19.83655163186929


### Calculate Unfrozen style encoder model's LSD (`model-leaky-relu-unfrozen-style-encoder-11-09-2023-2`)

In [19]:
import os

GENERATED_CUSTOM_ROOT = "/home/user/ss-vq-vae/experiments/outputs/random_audios_11-09-2023-2/"
with open("outputs/random_audios_11-09-2023-2/vqvae_list") as f:
    generated_custom_audios_list = f.read().splitlines()

lsds = []
for (_, (_, _, ground_path)), generated_path in zip(ground_audios_df.iterrows(), generated_custom_audios_list):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_CUSTOM_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    min_length = min(len(generated_audio), len(ground_audio))
    generated_audio = generated_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    lsds.append(eval_example_target(generated_audio, ground_audio)['lsd'])

print(sum(lsds) / len(lsds))

21.916483951838437


### Calculate Musaicing LSD

In [13]:
with open('outputs/random_audios/musaicing_list') as f:
    musaicing_audios_list = f.read().splitlines()
    
assert len(musaicing_audios_list) == len(ground_audios_df)

musaicing_lsds = []
for (_, (_, _, ground_path)), musaicing_path in zip(ground_audios_df.iterrows(), musaicing_audios_list):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    musaicing_path = os.path.join(GENERATED_ROOT, musaicing_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    musaicing_audio, _ = librosa.load(musaicing_path, sr=rate, mono=True)
    min_length = min(len(musaicing_audio), len(ground_audio))
    musaicing_audio = musaicing_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    musaicing_lsds.append(eval_example_target(musaicing_audio, ground_audio)['lsd'])

print(sum(musaicing_lsds) / len(musaicing_lsds))

15.367256811356247


### Calculate `cp-content` LSD

In [22]:
content_lsds = []
for _, (content_path, _, ground_path) in ground_audios_df.iterrows():
    content_path = os.path.join(HOME_ROOT, content_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    content_audio, _ = librosa.load(content_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(content_audio), len(ground_audio))
    content_audio = content_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    content_lsds.append(eval_example_target(content_audio, ground_audio)['lsd'])

print(sum(content_lsds) / len(content_lsds))

15.27539937605309


### Calculate `cp-style` LSD

In [23]:
style_lsds = []
for _, (_, style_path, ground_path) in ground_audios_df.iterrows():
    style_path = os.path.join(HOME_ROOT, style_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    style_audio, _ = librosa.load(style_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(style_audio), len(ground_audio))
    style_audio = style_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    style_lsds.append(eval_example_target(style_audio, ground_audio)['lsd'])

print(sum(style_lsds) / len(style_lsds))

17.8309627062074


## Calculate pitch set

In [10]:
import essentia.standard as estd
import pretty_midi

def get_pitches(audio):
    input_sr, sr = rate, 8000  # Need to resample because of EqualLoudness
    audio = estd.Resample(inputSampleRate=input_sr, outputSampleRate=sr)(audio)
    audio = estd.EqualLoudness(sampleRate=sr)(audio)
    rng = np.random.default_rng(seed=(audio > 0).sum())
    audio = rng.normal(loc=audio, scale=1e-4).astype(audio.dtype)  # To prevent Melodia from crashing
    pitches = estd.MultiPitchMelodia(sampleRate=sr)(audio)
    pitches = [[pretty_midi.utilities.hz_to_note_number(p) for p in pl if not np.isclose(0, p)]
               for pl in pitches]
    pitches = [[int(p + 0.5) for p in pl] for pl in pitches]
    return pitches

def eval_example_content(output, reference):
    pitches_output, pitches_reference = get_pitches(output), get_pitches(reference)
    assert len(pitches_output) == len(pitches_reference)
    jaccard = []
    for pl_output, pl_reference in zip(pitches_output, pitches_reference):
        matches = len(set(pl_output) & set(pl_reference))
        total = len(set(pl_output) | set(pl_reference))
        if total == 0:
            jaccard.append(0)
        else:
            jaccard.append(1 - matches / total)
    jaccard = np.mean(jaccard)
    return {'pitch_jaccard': jaccard}

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


### Calculate pitch jaccard for the model

In [11]:
import os
from tqdm import tqdm

pitches = []
for (_, (_, _, ground_path)), generated_path in tqdm(zip(ground_audios_df.iterrows(), generated_audios_list)):
    ground_path = os.path.join(HOME_ROOT, ground_path)
    generated_path = os.path.join(GENERATED_ROOT, generated_path)
    
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    generated_audio, _ = librosa.load(generated_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(ground_audio), len(generated_audio))
    ground_audio = ground_audio[:min_length]
    generated_audio = generated_audio[:min_length]

    pitches.append(eval_example_content(generated_audio, ground_audio)['pitch_jaccard'])

print(sum(pitches) / len(pitches))

721it [04:32,  2.65it/s]

0.9435879184078675


### Calculate pitch jaccard for `cp-content`

In [13]:
content_pitches = []
for _, (content_path, _, ground_path) in tqdm(ground_audios_df.iterrows()):
    content_path = os.path.join(HOME_ROOT, content_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    content_audio, _ = librosa.load(content_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(content_audio), len(ground_audio))
    content_audio = content_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    content_pitches.append(eval_example_content(content_audio, ground_audio)['pitch_jaccard'])

print(sum(content_pitches) / len(content_pitches))

721it [03:49,  3.14it/s]

0.7555776170025211


### Calculate pitch jaccard for `cp-style`

In [14]:
style_pitches = []
for _, (_, style_path, ground_path) in tqdm(ground_audios_df.iterrows()):
    style_path = os.path.join(HOME_ROOT, style_path)
    ground_path = os.path.join(HOME_ROOT, ground_path)

    style_audio, _ = librosa.load(style_path, sr=rate, mono=True)
    ground_audio, _ = librosa.load(ground_path, sr=rate, mono=True)
    # TODO: why do lengths differ and how should we handle it?
    min_length = min(len(style_audio), len(ground_audio))
    style_audio = style_audio[:min_length]
    ground_audio = ground_audio[:min_length]

    style_pitches.append(eval_example_content(style_audio, ground_audio)['pitch_jaccard'])

print(sum(style_pitches) / len(style_pitches))

721it [04:06,  2.92it/s]

0.9163400687308401
